# ENCODE bigWig analysis questions

Do your best to answer all parts of each question. You are encouraged to collaborate, but should turn in your own answers. 

Please limit each answer to a maximum of one markdown cell, one code cell and one plot. 

Put helper functions into a separate script (e.g. `hwutils.py`) so the notebook can be focused on plotting. Also see the [workshop on Clean Code](https://drive.google.com/file/d/1TraVwRkbkCbHq-s_-NS69ZEbRNwH8XNh/view) from Dan Larremore (https://larremorelab.github.io/slides/) for good coding tips to use in this assignment.


I worked with Allen and Yumin (in limited extent).

In [ ]:
# useful libraries to import

import pandas as pd
import numpy as np

import  sklearn.decomposition
from sklearn import manifold

import matplotlib.pyplot as plt
from matplotlib.ticker import EngFormatter
bp_formatter = EngFormatter('b') 
# nice way to format ticks as human-readable: ax.xaxis.set_major_formatter(bp_formatter)

from hwutils import *

In [ ]:
# load dataFrame of bigWigs from ENCODE (encodeproject.org/), binned to 10kb resolution across chromosome 10.
# note that the first three columns are chrom,start,end and the other columns are labeled by bigWig file accession.
df = pd.read_table('./data/ENCODE_GRCh38_binned_subset.tsv')

# load metadata from ENCODE for bigwig files. 
# can be queried as follows: bigwig_metadata.query("`File accession`==@ df_column_name ")
bigwig_metadata = pd.read_table('./data/ENCODE_GRCh38_bigWig_metadata.tsv')

- After loading the data (above), and visualize some of the profiles. Why might many signals dip on chr10 at around 40Mb?


In [ ]:
plt.figure(figsize=(15, 7))

# Iterating through columns and plotting them
for col in df.columns[3:]:
    plt.plot(df['start'], df[col], label=col)

x_position = df.iloc[3999]['start']
plt.axvline(x=x_position, color='red', linestyle='--', linewidth=1.5)

plt.title('Signal Profiles on Chromosome 10')
plt.xlabel('Position on Chromosome 10')
plt.ylabel('Signal Value')

plt.show()


A1: I suspect that the region on chromosome 10 with a low ChIP-seq signal corresponds to the centromere. The centromeric region is typically devoid of protein binding relevant for transcriptional activities.

- Use scikit-learn to perform PCA, and make a scatterplot of PC1 vs PC2.


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

data_to_pca = df.drop(columns=['chrom', 'start', 'end']) 
data_transposed = data_to_pca.T

# Perform PCA
pca = PCA(n_components=2)  # We want the first two principal components
principal_components = pca.fit_transform(data_transposed)

# Convert the principal components to a DataFrame for easier plotting
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# Plotting PC1 vs PC2
plt.figure(figsize=(10, 7))
plt.scatter(pca_df['PC1'], pca_df['PC2'], edgecolor='k')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of ENCODE Data: PC1 vs PC2')
plt.grid(True)
plt.show()

- Try to use the experiment metadata to understand and remove outliers. Try labeling or coloring points by various metadata columns. Were any columns in the metadata useful for outlier removal? Note that `sklearn.preprocessing.LabelEncoder()` can be useful for transforming text strings to categories, and `plt.text` can be used to overlay labels with points.

In [ ]:
bigwig_metadata

A3. I think PCA with Biosample term name label is the most useful category. This represents the sample's cell type, and adheres well to the outlier patterns. 

- Which Assays or Experiment Targets show broad vs narrow patterns? Is this consistent across cell types? Does this relate to the patterns seen in PCA? One way to investigate the characteristic scale is by computing the autocorrelation.


A4. I'm not sure what it exactly mean by broad vs narrow patterns, but TF-ChIP-seq seem to show narrower (i.e., more clustered) pattern than the Histone-ChIP-seq. I think it is more or less consistent throughout cell type.
As for the experimental target, there seems to be a general clustering regardless of specific cell type the sample is coming from, except few examples such as H3K4me3.

- Which "Experiment Targets" (e.g. histone marks or transcription factors) for which cell types are nearby in this PC1 vs PC2 space? Do any of these proximities have plausible biological interpretations? For example, are any polycomb-related factors in proximity? Illustrate this in a plot.


A5. Yes. The polycomb-related factors, including the RBBP5 and EZH-2 are relatively closely located in the PCA clusters. 



- How much does preprocessing matter? Try normalizing the variance per track and see if you arrive at similar or distinct conclusions. Try removing the region on chr10 mentioned above. Note that `sklearn.preprocessing.StandardScaler` could be useful for preprocessing. 


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Preprocess the data: Remove non-numeric columns and standardize
data_to_pca = df.drop(columns=['chrom', 'start', 'end']) 
data_to_pca = data_to_pca.drop(range(3999, 4151))

data_standardized = StandardScaler().fit_transform(data_to_pca)
data_transposed = data_standardized.T

# Perform PCA
pca = PCA(n_components=2)  # We want the first two principal components
principal_components = pca.fit_transform(data_transposed)

# Convert the principal components to a DataFrame for easier plotting
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# Plotting PC1 vs PC2
plt.figure(figsize=(10, 7))
plt.scatter(pca_df['PC1'], pca_df['PC2'], edgecolor='k')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of ENCODE Data: PC1 vs PC2')
plt.grid(True)
plt.show()

It reduced the issues with the outlier we had previously. 

- How many PCs are needed to explain 90% of the variance in the data? Illustrate this with a scree plot (https://en.wikipedia.org/wiki/Scree_plot). 


In [ ]:
# Step 1: Fit PCA on the data without specifying the number of components
pca_full = PCA()
pca_full.fit(data_transposed)

# Step 2: Calculate the cumulative explained variance
explained_variance_ratio = pca_full.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# Step 3: Plot the scree plot
plt.figure(figsize=(12, 6))
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.5, align='center', label='individual explained variance')
plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='cumulative explained variance')
plt.axhline(y=0.9, color='r', linestyle='--', label="90% explained variance")
plt.xlabel('Principal Component Number')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')
plt.legend(loc='best')
plt.tight_layout()
plt.grid(True)
plt.show()

# Step 4: Find number of components needed for 90% variance
num_components_90_variance = np.where(cumulative_variance >= 0.9)[0][0]
print(f"To explain at least 90% of the variance, {num_components_90_variance} principal components are needed.")


- How different is the dimensionality reduction into two dimensions for PCA from that obtained using MDS (multi-dimensional scaling)? What methods could be used to determine the similarity? Illustrate with a plot.


In [ ]:
dissimilarity_matrix = np.sqrt(((data_transposed[:, None] - data_transposed) ** 2).sum(axis=2))
mds = manifold.MDS(n_components=2, dissimilarity='precomputed')
X_mds = mds.fit_transform(dissimilarity_matrix)

# Plot the MDS representation
plt.figure(figsize=(8, 6))
plt.scatter(X_mds[:, 0], X_mds[:, 1],color='red', marker='o')
plt.xlabel('MDS Dimension 1')
plt.ylabel('MDS Dimension 2')
plt.title('MDS')
plt.show()

- Would non-negative matrix factorization (https://en.wikipedia.org/wiki/Non-negative_matrix_factorization) be a useful method to use for this dataset? Why or why not?  (No plots needed for this question).


NMF works best with datasets that only have positive values. It breaks down data into simpler parts that are easier to understand. For the ENCODE dataset, if all the data points are positive and we're looking for simpler, combined representations, NMF could work. It can be more advantageous because it can show additive patterns in the data that PCA cannot. I don't think the data itself contains negative values, so it should be possible to use implement.